In [ ]:
from checkers.game import Game
import numpy as np


In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/home/vaibhav/nanogpt/ourGPT/checkersengine/checkers/checkers/checkers_test.csv')

# Print the DataFrame
print(df)
# Strip whitespace and then split on space
df['transcript'] = df['transcript'].str.strip().str.split()

def remove_dots(string_list):
    return [item for item in string_list if '.' not in item]
df['transcript'] = df['transcript'].apply(remove_dots)

print(df)
print(df['transcript'][0][-1])

def remove_lastmove(string_list):
    return string_list[:-1]

df['transcript'] = df['transcript'].apply(remove_lastmove)
print(df['transcript'][0][-1])

def multiple_moves(moves_split):
    new_moves = moves_split[:2]
    moves_split = moves_split[2:]
    for move in moves_split:
        new_moves.append(new_moves[-1])
        new_moves.append(move)
    return new_moves



legal_moves = 0
total_moves = 0
game_over = 0

max_lengthmove = -1
min_lengthmove = 100

for game in df['transcript']:
    checker = Game()
    #print(game)
    move_counter = 0
    for moves in game:
        if '-' in moves:
            moves_split = moves.split('-')
        else:
            moves_split = moves.split('x')
        if len(moves_split) == 2:
            total_moves += 1
            x = int(moves_split[0])
            y = int(moves_split[1])
            if [x,y] in checker.get_possible_moves():
                legal_moves +=1
                move_counter +=1
                checker.move([x,y])
            else:
                break
        else:
            moves_split = multiple_moves(moves_split) 
            itr_max = len(moves_split)
            total_moves+=int(itr_max/2)
            itr = 0
            while(itr<itr_max):
                x = int(moves_split[itr])
                y = int(moves_split[itr+1])
                if [x,y] in checker.get_possible_moves():
                    legal_moves +=1
                    move_counter +=1
                    checker.move([x,y])
                else:
                    break
                itr+=2
    if checker.is_over():
        game_over+=1
    if max_lengthmove < move_counter:
        max_lengthmove = move_counter
    if move_counter< min_lengthmove:
        min_lengthmove = move_counter


print(max_lengthmove)
print(min_lengthmove)
print(legal_moves/11303)

In [ ]:
#Filtering for checking valid moves.

import pickle
with open('/home/vaibhav/nanogpt/ourGPT/test_timedecode.pkl', 'rb') as f:
    loaded_list = pickle.load(f)

#print(loaded_list)  # Output: [1, 2, 3, 4, 5]

for i, _ in enumerate(loaded_list):
    loaded_list[i] = loaded_list[i].strip()
    loaded_list[i] = loaded_list[i].split()
    #Remove the last string as it would be either indicating victory or it would be incomplete.
    loaded_list[i] = loaded_list[i][:-1]
    #Remove the numbering of the moves
    loaded_list[i] = [item for item in loaded_list[i] if '.' not in item]

for _ in loaded_list:
    print((_))


#'8x15x24'

In [ ]:
from checkers.game import Game

def multiple_moves(moves_split):
    new_moves = moves_split[:2]
    moves_split = moves_split[2:]
    for move in moves_split:
        new_moves.append(new_moves[-1])
        new_moves.append(move)
    return new_moves


#loaded_list = [['11-15', '23-18', '8-11', '26-23', '9-14', '18', '24-19', '15x24', '22x15', '7-11', '28x9', '11x18', '30-26', '5x14', '26-22', '3-7', '22x15', '10x19', '23-18', '14x23', '27x18', '6-10', '15x6', '1x10', '31-26', '2-6', '26-23', '11-16', '21-17', '16-20', '23-18', '20-24', '18-14', '10-15', '14-10', '6x15', '17-14', '15-19', '14-9', '19x26', '28x19', '15x24', '27x20', '26-30', '20-16', '30-26', '16-11', '26-23', '11-8', '23-19', '8-3', '19-15', '3-7', '15-18', '7-11', '12-16', '11-15', '20-16']]

legal_moves = 0
total_moves = 0
game_over = 0
max_lengthmove = -1

for game in loaded_list:
    checker = Game()
    print(game)
    for moves in game:
        if '-' in moves:
            moves_split = moves.split('-')
        else:
            moves_split = moves.split('x')
        if len(moves_split) == 2:
            total_moves += 1
            x = int(moves_split[0])
            y = int(moves_split[1])
            if [x,y] in checker.get_possible_moves():
                legal_moves +=1
                checker.move([x,y])
            else:
                print(moves)
                break
        else:
            moves_split = multiple_moves(moves_split) 
            itr_max = len(moves_split)
            total_moves+=int(itr_max/2)
            itr = 0
            while(itr<itr_max):
                x = int(moves_split[itr])
                y = int(moves_split[itr+1])
                if [x,y] in checker.get_possible_moves():
                    legal_moves +=1
                    checker.move([x,y])
                else:
                    print(moves)
                    break
                itr+=2
    if checker.is_over():
        game_over+=1


print(f'{legal_moves} legal_moves')
print(f'{total_moves} total_moves')
print(f'{game_over} gameover')


In [2]:
from checkers.game import Game
import numpy as np
import random

def possible_capture_move(move):
    x = move[0]
    y = move[1]
    if x<y and (((x-1) // 4) + 2) == ((y-1) //4):
        return True
    elif ((x-1) // 4)  == (((y-1) //4) +2):
        return True
    else:
        return False


def synthetic_games(game, moves):
    
    game_string = ";"
    for i in range(1, moves+1):
        game_string +=  str(i) + ". "
        for _ in range(2):
            possible_moves = game.get_possible_moves()
            random_index = random.randint(0, len(possible_moves) - 1)
            random_element = possible_moves[random_index]
            x = random_element[0]
            y = random_element[1]
            game.move([x, y])
            if game.is_over():
                return game_string
            if possible_capture_move(random_element):
                game_string+= str(x) + 'x' + str(y)
                double_capture = game.get_possible_moves()
                if (len(double_capture) != 1 or double_capture[0][0] != y):
                    game_string+=" "
                else:
                    while (len(double_capture) == 1 and double_capture[0][0] == y):
                        game.move([double_capture[0][0],double_capture[0][1]])
                        game_string+= "x"+str(double_capture[0][1])
                        y = double_capture[0][1]
                        double_capture = game.get_possible_moves()
                        if game.is_over():
                            return game_string+" "
                    game_string+=" "  
            else:
                game_string+= str(x) + '-' + str(y) + " "
    return game_string



In [ ]:
game = Game()
print(synthetic_games(game, 20))

In [6]:
from tqdm import tqdm

with open('synthetic_data16M.txt', 'w') as file:
    for _ in tqdm(range(16000000)):
        game = Game()
        file.write(synthetic_games(game, 20) + '\n')


  0%|          | 33674/16000000 [10:04<79:33:20, 55.75it/s] 


KeyboardInterrupt: 

In [ ]:
game = Game()
game.move([11,15])
game.move([24,20])
game.move([8,11])
game.move([28,24])
game.move([4,8])
game.move([21,17])
game.move([9,14])
game.move([22,18])
game.move([15,22])
game.move([25,18])

In [ ]:
game.get_possible_moves()

In [ ]:
#my random games
#sample format
#;1. 11-15 23-18 2. 8-11 26-23 3. 10-14 30-26 4. 6-10 24-19 5. 15x24 27x20 6. 4-8 28-24 7. 12-16 22-17 8. 8-12 32-28 9. 10-15 17x10 10. 7x14 26-22 11. 2-6 24-19 12. 15x24 28x19 13. 6-10 31-26 14. 1-6 22-17 15. 11-15 18x11 16. 14-18 23x14x7 17. 16x23x30 7-2 18. 9-13 2x9 19. 13x22 25x18 20. 5x14x23 21-17 1/2-1/2                                                                                          

# "1. 12-16 21-17 2. 9-14 17-13 3. 8-12 13-9 4. 6x13 25-21 5. 11-15 30-25 6. 3-8 24-19 7. 15x24 28x19 8. 7-11 27-24 9. 2-7 23-18 10. 16x23x30 18x9"
# "1. 11-15 23-19 2. 9-14 27-23 3. 8-11 31-27 4. 5-9 22-18 5. 15x22x31 24-20 6. 31x24x15 28-24 7. 3-8 25-22 8. 12-16 32-27 9. 16-19 23-16 10. 1-5 16-12" 


In [ ]:
def print_board(board):
    board_print = np.zeros((8,8))
    move_translater = {
    1 : (7,6), 2 : (7,4), 3 : (7,2), 4 : (7,0),
    5 : (6,7), 6 : (6,5), 7 : (6,3), 8 : (6,1),
    9 : (5,6), 10: (5,4), 11: (5,2), 12: (5,0),
    13: (4,7), 14: (4,5), 15: (4,3), 16: (4,1),
    17: (3,6), 18: (3,4), 19: (3,2), 20: (3,0),
    21: (2,7), 22: (2,5), 23: (2,3), 24: (2,1),
    25: (1,6), 26: (1,4), 27: (1,2), 28: (1,0),
    29: (0,7), 30: (0,5), 31: (0,3), 32: (0,1)
    }
    text_list = {}
    for piece in game.board.pieces:
	    if piece.position not in text_list and piece.position != None:
		    text_list[piece.position] = piece.player                 
    
    for key in text_list:
        x, y = move_translater[key]
        board_print[x][y] = text_list[key]
    
    print(board_print)




In [ ]:
game = Game()
singlegame = ";1. 11-15 24-20 2. 8-11 28-24 3. 9-13 22-18 4. 15x22 25x18 5. 4-8 26-22 6. 10-14 18x9 7. 5x14 22-18 8. 1-5 18x9 9. 5x14 29-25 10. 11-15 24-19 11. 15x24 25-22 12. 24-28 22-18 13. 6-9 27-24 14. 8-11 24-19 15. 7-10 20-16 16. 11x20 18-15 17. 2-6 15-11 18. 12-16 19x12 19. 10-15 11-8 20. 15-18 21-17 21. 13x22 30-26 22. 18x27 26x17x10x1 0-1"
secondgame = ";1. 11-15 23-18 2. 8-11 26-23 3. 10-14 30-26 4. 7-10 24-19 5. 15x24 28x19 6. 2-7 22-17 7. 4-8 26-22 8. 11-16 27-24 9. 16-20 32-28 10. 20x27 31x24 11. 8-11 24-20 0-1"

In [ ]:
game = Game()
while(True):
    move = input()
    if move == "stop":
        break
    
    parser = move.split("-")
    x = int(parser[0])
    y = int(parser[1])
    game.move([x,y])


In [ ]:
sample1 = "1. 11-16 22-17 2. 8-11 17-13 3. 4-8 24-19 4. 16-20 23-18 5. 9-14 18x9 6. 5x14 25-22 7. 11-15 27-23 8. 15x24 28x19 9. 8-11 22-18 10. 11-15 18x11 11. 7x16 29-25 12. 3-7 25-22 13. 7-11 22-18 14. 1-5 18x9 15. 5x14 26-22 16. 11-15 32-28 17. 15x24 28x19 18. 2-7 22-18 19. 14-17 21x14 20. 10x17 18-15 21. 20-24 15-11 22. 7-10 11-7 0-1"
sample2 = "1. 10-15 24-20 2. 7-10 28-24 3. 3-7 23-19 4. 9-14 22-17 5. 5-9 17-13 6. 11-16 20x11 7. 7x16x23 26x19 8. 14-18 30-26 9. 9-14 26-23 10. 8-11 25-22 11. 18x25 29x22 12. 1-5 23-18 13. 14x23 27x18 14. 11-16 18x11 15. 16x23 24-19 16. 23-26 32-27 17. 26-30 27-23 18. 10-14 22-17 19. 14-18 23x14 20. 6-9 13x6 21. 2x9x18 19-15 22. 30-25 15-10 23. 18-23 11-7 24. 5-9 17-13 25. 9-14 7-2 1/2-1/2"